# EC2 Setup - Terraform Fullstack

### Introduction

Ok, so in the previous lessons, we have prepared our codebase for Docker, and Dockerized our api.  We also wrote terraform scripts that would setup our rds instance and give it the appropriate permissions to access our ec2 instance.

But there is still more to do.  Namely we'll need to do provision our ec2 machine to automatically install docker, start docker, and pull down our images and boot them up properly.  

Let's get started.

### Setting Up Our EC2 Machine

Ok, so if we set up our infrastructure from the last lab by running terraform apply, we can then begin figuring out how to set up our ec2 machine.

We can do this by first sshing into our EC2 machine to make sure that each step works, and then we can setup terraform to automatically run each step.

So begin by sshing into the machine and confirm that our setup of psql works fine.

```bash
sudo apt-get update
sudo apt install postgresql-client-common
sudo apt-get install postgresql-client
```

Now the next step we can do is work on getting docker installed.

* `sudo apt-get install docker -y`

This looked like it worked, but if we run, `docker --version` to confirm it's installed, we see that it isn't available.  

So let's try a different way of installing docker.

`sudo snap install docker`

And then you should be able to start the docker service with the following:

`sudo systemctl start snap.docker.dockerd.service`

### Encoding our changes

Now before we keep going, remember that we'll want to write down each of these steps as code.  But we **should not** do this directly in a `.tf` file.  We'll have too many steps -- and it will grow to be too long.

Instead, copy over the steps to the `ec2-setup.sh` file.

If you look in the file, you'll see that we already got you started.

```bash
sudo apt-get update
sudo apt install postgresql-client-common
sudo apt-get install postgresql-client
```

So now it's your turn to copy over the steps of:

* Downloading docker with snap
* Starting the docker service with `systemctl`

### Keep Going

So let's keep up with this process of trying out a command in our ec2 instance, and then copying it over to the `ec2-setup.sh` file.

So next, pull the backend image that you uploaded to dockerhub previously.  And run the container, passing through the environmental variables that connect it to the database.  Specify the correct platform if you see an error.

Also, when running the container, give the container a name like `backend`.  You can do so with the `--name` argument.

* You can confirm it works, if get back some json by running the following:
    
`curl localhost:5000/positions`

And if each of these steps work, copy them over one by one, into the `ec2-setup.sh` file.

> Leave out each of the sudo's when copying it over.  If you don't, it may lead to a docker bug on your laptop.  Sounds painful.

### Working with our ec2-setup.sh file

* Trying it locally

Ok, so now let's see if these setup steps worked properly.  We can run the file on our laptop if we first comment out the lines that install psql and docker:

```
# sudo apt-get update
# sudo apt install postgresql-client-common
# sudo apt-get install postgresql-client
# sudo snap install docker
# sudo systemctl start snap.docker.dockerd.service
```

And then we can try it out by running the following on our laptop:

```bash
sh ec2-setup.sh
```

And once again confirm that you can connect to the `localhost:5000/positions` and get back the json.

```bash
curl localhost:5000/positions
```

> You may have to run the container in the background as a daemon, to then also make the curl request.  Ask chatgpt how to do this if you don't know how.

* Fixing a bug

Now one issue we may run into is, if we re-run the `ec2-setup.sh` file, we may see that we cannot re-boot up a container named `backend` (or anything else).  We cannot do this until we first remove the original `backend` container.

So we should update our script.  And we can update our `ec2-setup.sh` script to stop any running backend container, before creating a new one.  Here's what we can add to do so:

```bash
if docker ps -a | grep -wq backend; then
    echo "Stopping and removing backend..."
    docker stop backend
    docker rm backend
fi
```

Let's understand what the above is doing:

* `docker ps -a`: lists all running containers
* | grep -wq backend: searches that output of the running containers for the word backend end
* `if`, `fi`: This is our if statement in bash.  

So here we are saying if we find a match for a container called backend, then stop the container and remove it.

Add the if block before the `docker run` line and then confirm you can run the newly updated `ec2-setup.sh`file.

* Refactoring with variables

Ok, now that we have gotten these steps working, let's refactor our script with some variables.  So declare the following variables:

```bash
export BACKEND_CONTAINER="backend"
export BACKEND_IMAGE=""
export DB_USERNAME=""
export DB_PASSWORD=""
export DB_NAME=""
export DB_HOST="your_rds_instance_public_dns"
```

And then you can reference them later on with something like `${BACKEND_CONTAINER}`.  

> Note: you can also reference them with just `$BACKEND_CONTAINER`, but doing it the way we recommended will make things easier later on.

Ok, so once you have declared and referenced each of the above variables, re-run:

```bash
sh ec2-setup.sh
```

And confirm that the script properly boots up the flask application.

### Moving to EC2

Now that we have the `ec2-setup.sh` script working locally, the next step is to try it out on our ec2 instance.  To accomplish that, let's uncomment anything that we have initially commented out (ie. all the installation steps should be uncommented).  And also make sure that the `DB_HOST` properly points to the domain name of the rds instance.  You may also need to specify to run the docker container with the right platform, `platform=linux/amd64/v2`.

So `scp` the file over to the ec2 instance (ask chatgpt if you forget how), and then run the file on the ec2 machine.

> **Note:** We'll need to run the script with sudo, so you can do so with:

```
sudo sh ec2-setup.sh
```

Ok, so confirm that you can curl to `localhost:5000/positions`.  

> If you cannot, read the error message, and confirm that your environmental variables are properly set up.

### Automating with Templates

> **Warning**: Potential setup issue with templates on mac: So if you run into an error, [please see the following](https://discuss.hashicorp.com/t/template-v2-2-0-does-not-have-a-package-available-mac-m1/35099/3).

Now if scping and running the `ec2-setup.sh` script worked ok, how can we use terraform to automatically get that file onto our ec2 machine?

We can do so with a feature called templates.

With templates we can copy over all of the contents of our `ec2-setup.sh` file, into our `ec2-setup.tpl` file, which you'll see inside of the `ec2-setup.tpl` file.

```bash
sudo apt-get update
sudo apt install postgresql-client-common
sudo apt-get install postgresql-client -y
sudo snap install docker
sudo systemctl start snap.docker.dockerd.service

if sudo docker ps -a | grep -wq ${BACKEND_CONTAINER}; then
    echo "Stopping and removing ${BACKEND_CONTAINER}..."
    sudo docker stop ${BACKEND_CONTAINER}
    sudo docker rm ${BACKEND_CONTAINER}
fi

sudo docker run -d -p 5000:5000 --name ${BACKEND_CONTAINER} -e DB_USERNAME=${DB_USERNAME} -e DB_PASSWORD=${DB_PASSWORD} -e DB_NAME=${DB_NAME} -e DB_HOST=${DB_HOST} --platform=linux/amd64/v2 ${BACKEND_IMAGE}
```

> So notice we are back to preceding each docker command with `sudo`.

Then in the `web_app.tf` file, you'll add another resource that looks like the following:

```bash
data "template_file" "init_script" {
  template = file("${path.module}/ec2-setup.tpl")
    
  vars = {
    BACKEND_CONTAINER=""
    BACKEND_IMAGE=""
    DB_USERNAME=""
    DB_PASSWORD=""
    DB_NAME=""
    DB_HOST=""
  }
}
```

Add this to the `web_app.tf` file, update the variable name values from strings to the appropriate values. 

> For the `DB_HOST`, this should be calculated by getting the `endpoint` property from the database resource in terraform.  You can figure out how to do this with the terraform console if need be.  

Ok, so let's get a sense of how a template works.  So templates are a data element, and we define them by specifying:

* The location of the template.  The `${path.module}` returns the path where the `terraform` folder is located on your computer.
* The variables to pass through to the template.

So instead of declaring the variables up top with `BACKEND_CONTAINER="backend"` we just do so in our `.tf` file.

Now the template actually, takes the variables and the `.tpl` file and compiles this into text.  You can see this if you login to the console:

`terraform console`

And then reference the template.

```bash
data.template_file.init_script.rendered
```

So the above line specifies the `data.template_file` resource, and the name of our resource `init_script`, and then shows the compiled text with a call to `.rendered`.

Ok, so we can use the template to spit out some text, using the `rendered` property.  And where should we place this text to make sure it is run in our ec2 machine?  In the `user_data` property.  

So update your `aws_instance` resource to look like the following:

```bash
resource "aws_instance" "backend_server" {
  ami           = "ami-07d9b9ddc6cd8dd30"
  instance_type = "t2.micro"
  key_name = "example"
  # 
  depends_on = [aws_db_instance.postgres_db]
  vpc_security_group_ids = [aws_security_group.web_app.id]
  
  user_data = data.template_file.init_script.rendered
  
  tags = {
      Name = "backend server"
  }
}
```

Ok, so now let's reapply our terraform code, and see if it ran our template.

> It will likely break.

### Debugging

Let's ssh into the ec2 machine and see how far we got.  You can get a sense of where our script worked, and where it broke by checking the following:

* Is there a container running: `sudo docker ps`
* Did our image get pulled down: `sudo docker image ls`
* Is docker even installed?: `sudo docker --version`


And then remember that we can see the logs of what occurred with the following:

```bash
cat /var/log/cloud-init-output.log
cat /var/log/cloud-init.log
```

What often occurs is that our script will proceed with the next step, before the previous one has finished. To prevent this from occurring, we can add some `while` blocks.  For example, let's update the start of our `ec2-setup.tpl` file to the following:

```bash
sudo apt-get update
sudo apt install postgresql-client-common
sudo apt-get install postgresql-client -y
sudo snap install docker
sudo systemctl start snap.docker.dockerd.service

while ! sudo docker ps; do
    echo 'Waiting for docker to be ready'
    sudo snap install docker
    sudo systemctl start snap.docker.dockerd.service
    sleep 5
done
```

So in the `while` block, we are saying, when we don't get a back a value from running `docker ps` (ie when we get an error from running `docker ps`), try to re-download and restart docker, and then wait a few seconds before checking again.

This while block will ensure that we don't move onto the next steps, before docker is running.

* Your turn

Ok, so now let's add a few more while blocks to our `ec2-setup.tpl` file.

1. Instead of the line `sudo docker pull ${BACKEND_IMAGE}`, change this so that the logic is:
    * When we don't get a positive value from looking for our backend image, repull that image down and then sleep for 5 seconds
    
2. Instead of just running our container at the very end, wrap this in a while block that says: 
* if we don't see the container running, re-run the container.

To get a sense of if your code works, you can first try it out locally in your `ec2-setup.sh` file.  Then of course update the `ec2-setup.tpl` file.  And then re-apply the changes, and sh into your ec2 instance and then make the curl request to confirm that it properly returns the json.

* If you run into errors, follow the debugging tips that we listed above.

### Summary

Ok, so in this lesson, we learned about coding bash scripts, which we can test out locally.  And then we learned to translate these bash scripts into templates that automatically run when initializing our ec2 instance.

Along the way, we learned about coding while loops in bash:

```bash
while ! sudo docker ps; do
    echo 'Waiting for docker to be ready'
    sudo snap install docker
    sudo systemctl start snap.docker.dockerd.service
    sleep 5
done
```

And if blocks:

```bash
if sudo docker ps -a | grep -wq ${BACKEND_CONTAINER}; then
    echo "Stopping and removing ${BACKEND_CONTAINER}..."
    sudo docker stop ${BACKEND_CONTAINER}
    sudo docker rm ${BACKEND_CONTAINER}
fi
```

We also practiced debugging on our ec2 instance by inspecting the log files.

```bash
cat /var/log/cloud-init-output.log
cat /var/log/cloud-init.log
```

### Resources

[Terraform working with Following](https://discuss.hashicorp.com/t/template-v2-2-0-does-not-have-a-package-available-mac-m1/35099/3)